In [1]:
from keras.models import Sequential, Model, load_model
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.engine.topology import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.applications.resnet50 import ResNet50
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# from image_util import crop_center
import pickle as pkl
import matplotlib.pyplot as plt
# from keras.utils.visualize_util import plot
import os
%matplotlib inline

Using TensorFlow backend.
/Users/mbaltac/anaconda/envs/dl_env/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
# load data
Xtrain = np.load('./data/dataset256/Xtrain.npy')
Xtest = np.load('./data/dataset256/Xtest.npy')
Ytrain_reg = np.load('./data/dataset256/Ytrain_reg.npy')
Ytest_reg = np.load('./data/dataset256/Ytest_reg.npy')
Ytrain_cls = np.load('./data/dataset256/Ytrain_cls.npy')
Ytest_cls = np.load('./data/dataset256/Ytest_cls.npy')

In [3]:
print ('Xtrain', Xtrain.shape)
print ('Xtest', Xtest.shape)
print ('Ytrain_reg', Ytrain_reg.shape)
print ('Ytest_reg', Ytest_reg.shape)
print ('Ytrain_cls', Ytrain_cls.shape)
print ('Ytest_cls', Ytest_cls.shape)

Xtrain (15405, 256, 256, 3)
Xtest (3852, 256, 256, 3)
Ytrain_reg (15405,)
Ytest_reg (3852,)
Ytrain_cls (15405,)
Ytest_cls (3852,)


In [4]:
# netowrk settings

batch_size = 64
folder = './myresnet-training-checkpoint-new/'
if not os.path.exists(folder):
    os.makedirs(folder)

n_cls = int(np.max(Ytrain_cls))


In [24]:
Ytrain_cls = to_categorical(Ytrain_cls-1)
Ytest_cls = to_categorical(Ytest_cls-1)

In [6]:
def addResidualBlock(inp, size=(3,3), subsample=(4,4), filters_conv_rate=2, name=None):
    conv_filters = int(inp.get_shape()[-1])*filters_conv_rate
    inp = BatchNormalization()(inp)
    first = Convolution2D(conv_filters, size[0], size[1], border_mode='same', subsample=subsample, activation='relu')(inp)
    
    second = BatchNormalization()(first)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    second = BatchNormalization()(second)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    return merge([first, second], mode='sum', name=name)

In [16]:
def MyResNetRegr():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(1, name='out')(out)
    
    model = Model(input=inp, output=out)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def MyResNetCls():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(n_cls, activation='softmax', name='out')(out)
    
    model = Model(input=inp, output=out)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


def MyResNet2Obj():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out_reg = Dense(1, name='out_reg')(out)
    out_cls = Dense(n_cls, activation='softmax', name='out_cls')(out)
    
    model = Model(input=inp, output=[out_reg, out_cls])
    mr = Model(input=inp, output=out_reg)
    mc = Model(input=inp, output=out_cls)

    model.compile(loss=['mean_squared_error', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    mr.compile(loss='mean_squared_error', optimizer='adam')
    mc.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model, mr, mc


In [17]:
load_path = False

In [18]:
# load_path = './myresnet-training-checkpoint/reg_checkpoint/checkpoint-99-0.00.hdf5'
if load_path:
    model_reg = load_model(load_path)
else:
    model_reg = MyResNetRegr()
# model_reg.summary()

In [19]:
# load_path = './myresnet-training-checkpoint/cls_checkpoint/checkpoint-99-0.39.hdf5'
if load_path:
    model_cls = load_model(load_path)
else:
    model_cls = MyResNetCls()
# model_cls.summary()

In [20]:
# load_path = './myresnet-training-checkpoint/2obj_checkpoint/checkpoint-99-1.55.hdf5'
if load_path:
    model = load_model(load_path)
else:
    model, mr, mc = MyResNet2Obj()
# model.summary()

In [26]:
if not os.path.exists(folder + '2obj_checkpoint/'):
    os.makedirs(folder + '2obj_checkpoint/')
checkpoint = ModelCheckpoint(filepath=folder + '2obj_checkpoint/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
history = model.fit(x=Xtrain[:100], y=[Ytrain_reg[:100], Ytrain_cls[:100]], 
          nb_epoch=2, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 
mr.save(folder + '2obj_checkpoint/model_reg.hdf5')
mc.save(folder + '2obj_checkpoint/model_cls.hdf5')
with open(folder + '2obj_checkpoint/history.pkl', 'wb') as f:
    pkl.dump(history.history, f)

Train on 90 samples, validate on 10 samples
Epoch 1/2
90/90 [==============================] - 1s - loss: 1.9922 - out_reg_loss: 0.2892 - out_cls_loss: 1.7030 - out_reg_acc: 0.0000e+00 - out_cls_acc: 0.4000 - val_loss: 37.8475 - val_out_reg_loss: 31.5371 - val_out_cls_loss: 6.3103 - val_out_reg_acc: 0.0000e+00 - val_out_cls_acc: 0.3000
Epoch 2/2
90/90 [==============================] - 1s - loss: 2.1073 - out_reg_loss: 0.5001 - out_cls_loss: 1.6073 - out_reg_acc: 0.0000e+00 - out_cls_acc: 0.3778 - val_loss: 15.0120 - val_out_reg_loss: 8.5387 - val_out_cls_loss: 6.4733 - val_out_reg_acc: 0.0000e+00 - val_out_cls_acc: 0.4000


In [27]:
if not os.path.exists(folder + 'reg_checkpoint/'):
    os.makedirs(folder + 'reg_checkpoint/')
checkpoint = ModelCheckpoint(filepath=folder + '/reg_checkpoint/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
history = model_reg.fit(x=Xtrain[:100], y=Ytrain_reg[:100], 
          nb_epoch=2, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 
with open(folder + 'reg_checkpoint/history.pkl', 'wb') as f:
    pkl.dump(history.history, f)

Train on 90 samples, validate on 10 samples
Epoch 1/2
90/90 [==============================] - 7s - loss: 1.1124 - val_loss: 45.6573
Epoch 2/2
90/90 [==============================] - 1s - loss: 1.2440 - val_loss: 4.5591


In [29]:
if not os.path.exists(folder + 'cls_checkpoint/'):
    os.makedirs(folder + 'cls_checkpoint/')
checkpoint = ModelCheckpoint(filepath=folder + '/cls_checkpoint/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
history = model_cls.fit(x=Xtrain[:100], y=Ytrain_cls[:100], 
          nb_epoch=2, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 
with open(folder + 'cls_checkpoint/history.pkl', 'wb') as f:
    pkl.dump(history.history, f)

Train on 90 samples, validate on 10 samples
Epoch 1/2
90/90 [==============================] - 8s - loss: 2.2229 - val_loss: 4.0512
Epoch 2/2
90/90 [==============================] - 1s - loss: 1.6137 - val_loss: 4.4345


In [ ]:
evl_both = model.evaluate(x=Xtest, y=[Ytest_reg, Ytest_cls],batch_size=batch_size)

In [ ]:
evl_cls = model_cls.evaluate(x=Xtest, y=Ytest_cls,batch_size=batch_size)

In [ ]:
evl_reg = model_reg.evaluate(x=Xtest, y=Ytest_reg,batch_size=batch_size)

In [ ]:
model.metrics_names

In [ ]:
evl_both

In [ ]:
evl_cls

In [ ]:
evl_reg

In [39]:
with open(folder + 'reg_checkpoint/history.pkl', 'rb') as f:
    hitsory = pkl.load(f) 


In [41]:
print(hitsory)

{'loss': [1.1123580733935039, 1.2439877324634128], 'val_loss': [45.657299041748047, 4.5590920448303223]}


In [36]:

plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable